In [1]:
import os

In [2]:
%pwd

'd:\\Version Control\\AI-Text-Summarizer\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\Version Control\\AI-Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    model_path: Path
    data_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)


        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation


        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path,
            tokenizer_path=config.tokenizer_path,
            metric_file_name=config.metric_file_name
        )

        return model_evaluation_config

In [8]:
from datasets import load_dataset, load_from_disk
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import evaluate
from tqdm import tqdm
import torch

# load rouge metric
metric = evaluate.load("rouge")


c:\Users\aminu\miniconda3\envs\textS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config


    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        """Yield successive n-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i:i + batch_size]

    def calculate_metric_on_test_ds(self, dataset, metric, model, tokenizer,batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu",
                                column_text="article", column_summary="highlights"):
        
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):

            inputs = tokenizer(article_batch, max_length=1024, truncation=True, padding="max_length", return_tensors="pt")

            summeries = model.generate(input_ids=inputs["input_ids"].to(device),
                                      attention_mask=inputs["attention_mask"].to(device),
                                      max_length=128,
                                      num_beams=8,
                                      length_penalty=0.8)

            decoded_summeries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for s in summeries]
            decoded_summeries = [d.replace("", " ") for d in decoded_summeries]

            metric.add_batch(predictions=decoded_summeries, references=target_batch)

            score = metric.compute()
            return score

    def evaluate(self):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)

        dataset_samsum_pt = load_from_disk(self.config.data_path)
        
        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

        rouge_metric = evaluate.load('rouge')

        score = self.calculate_metric_on_test_ds(
            dataset_samsum_pt['test'][0:10],
            rouge_metric,
            model_pegasus,
            tokenizer,
            batch_size=2,
            column_text="dialogue",
            column_summary="summary")
        
        rouge_dict = dict((rn, score[rn]) for rn in rouge_names)

        df = pd.DataFrame(rouge_dict, index=['pegasus'])
        df.to_csv(self.config.metric_file_name, index=False)

In [15]:
try:
    Config = ConfigurationManager()
    model_evaluation_config = Config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception as e:
    raise e

[2025-10-24 00:26:30,685: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-24 00:26:30,689: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-24 00:26:30,692: INFO: common: created directory at: artifacts]
[2025-10-24 00:26:30,694: INFO: common: created directory at: artifacts/model_evaluation]


  0%|          | 0/5 [00:00<?, ?it/s]

[2025-10-24 00:27:07,269: INFO: rouge_scorer: Using default tokenizer.]


  0%|          | 0/5 [00:34<?, ?it/s]
